# Regression Predict Student Solution

© Explore Data Science Academy

---
### Honour Code

I {**YOUR NAME, YOUR SURNAME**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Predict Overview: Spain Electricity Shortfall Challenge

The government of Spain is considering an expansion of it's renewable energy resource infrastructure investments. As such, they require information on the trends and patterns of the countries renewable sources and fossil fuel energy generation. Your company has been awarded the contract to:

- 1. analyse the supplied data;
- 2. identify potential errors in the data and clean the existing data set;
- 3. determine if additional features can be added to enrich the data set;
- 4. build a model that is capable of forecasting the three hourly demand shortfalls;
- 5. evaluate the accuracy of the best machine learning model;
- 6. determine what features were most important in the model’s prediction decision, and
- 7. explain the inner working of the model to a non-technical audience.

Formally the problem statement was given to you, the senior data scientist, by your manager via email reads as follow:

> In this project you are tasked to model the shortfall between the energy generated by means of fossil fuels and various renewable sources - for the country of Spain. The daily shortfall, which will be referred to as the target variable, will be modelled as a function of various city-specific weather features such as `pressure`, `wind speed`, `humidity`, etc. As with all data science projects, the provided features are rarely adequate predictors of the target variable. As such, you are required to perform feature engineering to ensure that you will be able to accurately model Spain's three hourly shortfalls.
 
On top of this, she has provided you with a starter notebook containing vague explanations of what the main outcomes are. 

<a id="cont"></a>

## Table of Contents

<a href=#one>1.Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Cleaning Data</a>

<a href=#four>4. Exploratory Data Analysis (EDA)</a>

<a href=#five>5. Data Engineering</a>

<a href=#six>6. Modeling</a>

<a href=#seven>7. Model Performance</a>

<a href=#eight>8. Model Explanations</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

In [97]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Libraries for data preparation and model building
import scipy as sp
import statsmodels as sm
import sklearn.model_selection as skl


# Setting global constants to ensure notebook results are reproducible
PARAMETER_CONSTANT = 42

<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

In [ ]:
df = pd.read_csv('df_train.csv')
df_test = pd.read_csv('df_test.csv')

<a id="three"></a>
## 3. Data Cleaning
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>


In [ ]:
df.head()

##### The column `Unnamed: 0` is dropped becaues it is an exact replica of the index column.

In [ ]:
df = df.drop("Unnamed: 0", axis =1)

##### The dataset has `49 features` and `8763 observations`.

In [ ]:
df.shape

##### There are `2068 null entries` in total in the entire dataset.

In [ ]:
# Checks for total nulls in the dataframe
df.isnull().sum().sum()

##### The function `check_nulls` tallies the number of nulls per column.

In [ ]:
#  Checks for total per column
def check_nulls (data):
    null_dict = {}

    for col in df.columns:

        if df[col].size > df[col].count():
            nulls = df[col].size - df[col].count()
            percent_missing = round(nulls / df[col].size * 100)
            null_dict[col] = f'has {nulls} nulls making up {percent_missing}% missing values'
    return null_dict

##### Running `check_nulls` exposes that all null values are in one colunm. Namely, `valencia_pressure`.

In [ ]:
check_nulls(df)

##### The column names are all cast to lower case for ease of referencing in the later phases.
##### The columns are set in alphabetical order so that the features are grouped by city.
##### The time feature is kept in the first index position.

In [ ]:
# 1. Order the columns in alphabetical order
df = df.reindex(sorted(df.columns), axis=1)

# 2. Keep the "time" column in the first index position
df = df[['time'] + [col for col in df.columns if col != 'time']]

# 3. Keep the "load_shortfall_3h" column last
df = df[[col for col in df.columns if col != 'load_shortfall_3h'] + ['load_shortfall_3h']]

# 4. Convert all column titles to lowercase
df.columns = df.columns.str.lower()

df.head()

<a id="four"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

##### The `load_shortfall_3h` column name is stored in a `target_variable`.

In [ ]:
target_variable = 'load_shortfall_3h'

##### The function `get_categorical_cols` outputs a list of all the columns that are of `object` datatype.
##### Other than `time`, there are two others. `seville_pressue` and `valencia_wind_deg`.

In [ ]:
# outputs dataframe of catagorical data
def get_categorical_cols(df):
    non_numeric = []
    for col in df.columns:
        if df[col].dtype == object:
          non_numeric.append(col)
    return df[non_numeric]

categorical_df = get_categorical_cols(df)
categorical_df.head()

In [ ]:
categorical_df.shape

##### The function `get_numeric_cols` outputs a list of all the columns that are of `float` or `int` datatype values.

In [ ]:
def get_numeric_cols(df):
    non_numeric = []
    for col in df.columns:
        if df[col].dtype != object:
          non_numeric.append(col)
    return df[non_numeric]

numeric_df = get_numeric_cols(df)
numeric_df.head()

In [ ]:
numeric_df.shape

##### The kurtosis of all columns is checked to figure out which columns possibly contain outliers. 

In [ ]:
kurtosis_values = df.kurtosis(numeric_only=True)
outlier_columns = kurtosis_values[kurtosis_values > 3].index
outlier_columns

# columns with potential outliers 
# to be normalisation later on

##### Using a for loop a list containing the names of the cites is extracted. 

In [ ]:
# list of unique city names 
unique_cities = []
for col in df.drop(['time', 'load_shortfall_3h'], axis=1).columns:
    city = col.split('_')[0]
    if city not in unique_cities:
        unique_cities.append(city)

unique_cities

##### Using `city_df` a unique dataset for all cities is created. This step is mostly for convenience so that more manageable correlation heatmaps can be created later on.

In [ ]:
# returns a dataframe with weather data for each ciy
def city_df (df, city_name):

    cities = {}

    for col in df.columns:
        city = col.split('_')[0]

        if city not in cities:
            cities[city] = []

            for col in df.columns:
                if city in col:
                    cities[city].append(col)
    
    cities[city_name].append(target_variable)
    
    return df[cities[city_name]]

### Number Summaries

##### The number summaries for the numeric columns reveal an interesting characteristic about all the features related to rain. The columns are mostly filled with `0` entries and have very low varience in their values. These columns are primary candidates for removal during Data Enginnering.

In [ ]:
# description of the barcelona weather values
barcelona_df = city_df(df, unique_cities[0])
barcelona_df.describe()

In [ ]:
# description of the barcelona weather values
bilbao_df = city_df(df, unique_cities[1])
bilbao_df.describe()

In [ ]:
# description of the madrid weather values
madrid_df = city_df(df, unique_cities[2])
madrid_df.describe()

In [ ]:
# description of the seville weather values
seville_df = city_df(df, unique_cities[3])
seville_df.describe()

In [ ]:
# description of the seville weather values
valencia_df = city_df(df, unique_cities[4])
valencia_df.describe()

##### A scatterplot of all the numeric features were plotted against the target feature `load_shortfall`. There were no obvious trends discernible and no feature stands out as either strongly or negatively correlated to the target.  

In [ ]:
# plot relevant feature interactions
index = 0
fig, axs = plt.subplots(15, 3, figsize=(15, 30))
for i in range(15):
    for j in range(3):
        var = numeric_df.columns[index]
        sns.scatterplot(data=numeric_df, x=var, y=target_variable, ax=axs[i,j])
        axs[i,j].set_title(f'{var} vs. {target_variable}')
        index += 1    
fig.tight_layout()
plt.show()

In [ ]:
categorical_df.shape

In [ ]:
# have a look at feature distributions
plt.figure(figsize =(32,9))
sns.histplot(df['load_shortfall_3h'], kde=True)
plt.xlabel('Load Shortfall')
plt.ylabel('Count')
plt.title('Distribution of load_shortfall_3h')
plt.show()

##### We used boxplot visuals to check for possible trends in the non-catagorical data. Nothing significant pops up for any one catagory. Their median values are rougly equivalent and so are their ranges. Although a there is significant enough visible variation in the interquatile ranges for the shortfall amount that this feature can be retained for inclusion in the model.

In [ ]:
for var in categorical_df.drop("time", axis=1).columns:
     plt.figure(figsize =(20,5))
     sns.boxplot(x=var, y=target_variable, data = df)
     plt.title(f'{var} vs. {target_variable}')
     plt.show()

##### To further explore relationships between features, we plotted correlation heatmaps for each city separately. We observed that although all the features fall within the rrange of weak to no correlation at all, temperature features overall seem to have the strongest relationship with load shortfall, followed by wind degree, which not surprisingly, has a negative correlation (althought still very weak) with the target variable. 

In [ ]:
# Select variables for correlation matrix
barca_corr_matrix = barcelona_df.corr(numeric_only=True)

# Plot the correlation matrix as a heatmap
plt.figure(figsize =(10,10))
sns.heatmap(barca_corr_matrix, annot=True, cmap='coolwarm', xticklabels=True, yticklabels=True)
plt.title('Barcelona Correlation Matrix')
plt.show()

In [ ]:
# Select variables for correlation matrix
sevi_corr_matrix = seville_df.corr(numeric_only=True)

# Plot the correlation matrix as a heatmap
plt.figure(figsize =(10,10))
sns.heatmap(sevi_corr_matrix, annot=True, cmap='coolwarm', xticklabels=True, yticklabels=True)
plt.title('Seville Correlation Matrix')
plt.show()

In [ ]:
# Select variables for correlation matrix
bilb_corr_matrix = bilbao_df.corr(numeric_only=True)

# Plot the correlation matrix as a heatmap
plt.figure(figsize =(10,10))
sns.heatmap(bilb_corr_matrix, annot=True, cmap='coolwarm', xticklabels=True, yticklabels=True)
plt.title('Bilboa Correlation Matrix')
plt.show()


In [ ]:
# Select variables for correlation matrix
madr_corr_matrix = madrid_df.corr(numeric_only=True)

# Plot the correlation matrix as a heatmap
plt.figure(figsize =(10,10))
sns.heatmap(madr_corr_matrix, annot=True, cmap='coolwarm', xticklabels=True, yticklabels=True)
plt.title('Madrid Correlation Matrix')
plt.show()

In [ ]:
# Select variables for correlation matrix
vale_corr_matrix = valencia_df.corr(numeric_only=True) 

# Plot the correlation matrix as a heatmap
plt.figure(figsize =(10,10))
sns.heatmap(vale_corr_matrix, annot=True, cmap='coolwarm', xticklabels=True, yticklabels=True)
plt.title('Valencia Correlation Matrix')
plt.show()

##### Lastly we took a look at how the shortfall changes over time. It revealed and interesting pattern that looks like it has fluctuations that are seasonal as well as nested cycles that occured at a higher frequency than the broader overall cycle. 

In [ ]:
plt.figure(figsize =(32,9))
plt.plot(np.arange(len(df[target_variable])), df[target_variable])
plt.title("load shortfall over time")
plt.xlabel("Time (3h)")
plt.ylabel("Shortfall")
plt.show()

##### To get clearer analysis on possible time related patterns in shortfall we created frequency tables to aggregate averages for daily and annual cycles. 

In [ ]:
hour_freqs = {}
for index, row in df.iterrows():
    hour = row["time"][11:]
    if hour in hour_freqs:
        hour_freqs[hour][0] += row[target_variable]
        hour_freqs[hour][1] += 1
    else:
        hour_freqs[hour] = [row[target_variable], 1]

##### The bar chart shows a very high average shortfall between 12am midday and 3pm and again 12 midnight and 3am for the day cycles. 

In [ ]:
# Extract x-values (hours) and y-values (quotients)
hr_x_values = list(hour_freqs.keys())
hr_y_values = [value[0] / value[1] for value in hour_freqs.values()]

# Plot the histogram
plt.figure(figsize=(16, 6))
plt.bar(hr_x_values, hr_y_values)
plt.xlabel('Hours')
plt.ylabel('Avarage Shortfall')
plt.title('Average Shortfall Per Three Hour Period')
plt.show()

##### Below is a frequency table created to check for patterns in the annual cycles. 

In [ ]:
mon_freqs = {}
for index, row in df.iterrows():
    mon = row["time"][5:7]
    if mon in mon_freqs:
        mon_freqs[mon][0] += row[target_variable]
        mon_freqs[mon][1] += 1
    else:
        mon_freqs[mon] = [row[target_variable], 1]

##### The bar chart confirms the initial observartion of a cycle within cycles of peaks and reduction in shortfall. There seems to be on average a higher shortfall between July and November and a lower avaerage shortfall between January and May. 

In [ ]:
# Extract x-values (hours) and y-values (quotients)
mn_x_values = list(mon_freqs.keys())
mn_y_values = [value[0] / value[1] for value in mon_freqs.values()]

mon_list = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"] 
# Customize the x-labels


# Plot the histogram
plt.figure(figsize=(16, 6))
plt.bar(mn_x_values, mn_y_values)
plt.xlabel('Hours')
plt.ylabel('Avarage Shortfall')
plt.title('Average Shortfall Per Three Hour Period')
plt.xticks(ticks=np.arange(12), labels=mon_list, rotation=45, ha='right', fontsize=10)
plt.show()

<a id="five"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

### Section Goals
##### - Create a new dataframe called `df_clean` for engineering purposes.
##### - Impute null values for the `valencia_pressue` feature.
##### - Remove `rain` and `snow` related columns.
##### - Minimize noise by selecting the relevant `temperature` related columns for each city.
##### - Convert  the `categorical` columns to `numerical` columns. 
##### - `Normalize` all the selected features to prepare them for model training. 
##### - Convert the `time` column to a `datetime` object.

In [ ]:
df_clean = df.copy()
df_clean.shape

##### Imputing `valencia_pressure`
##### The median value was selected for the column since it approximates the arithmetic average of the mode and mean values for the feature. 

In [ ]:
valencia_pressure_median = df_clean['valencia_pressure'].median()

In [ ]:
df_clean['valencia_pressure'] = df_clean['valencia_pressure'].fillna(valencia_pressure_median, axis=0)
df_clean['valencia_pressure'].isna().sum()

##### The next step is transforming the catagorical data into numerical format incase it is kept for training the model later on.

In [ ]:
for col in df_clean.columns:
    if df_clean[col].dtype == object and col != "time":
        df_clean[col]= df_clean[col].str.extract(r'([0-9]+)')
        df_clean[col] = pd.to_numeric(df_clean[col])

##### Removing `rain` and `snow` related columns. 
##### Since the rain and snow related columns are continuous measurement with little variation in value they are show little significanc as features to train the model on and as such are being dropped from the training data set. 

In [ ]:
cols_to_remove = []
for col in df_clean.columns:
    if "rain" in col or "snow" in col:
        cols_to_remove.append(col)

##### Temperature columns are highly correlated to each. Since the temp_min column for each city seems to be more correlated to the target in most cases than the other values, this is the one that will be kept for training the model. The rest will be discarded.

In [ ]:
for col in df_clean.columns:
    if "temp" in col and "min" not in col:
        cols_to_remove.append(col)

##### The rest of the columns are kept removed to preserve homogeny in the train dataset. All five cities have `pressure`, `wind speed`, and `temprature` in common, these seem like th best features to use for training the model since all three factors apply to all of Spain. It is hoped that they could be used to give a good overall picture on how weather impacts energy shortfall. 

In [ ]:
for col in df_clean.columns:
    if "clouds" in col:
        cols_to_remove.append(col)

In [ ]:
for col in df_clean.columns:
    if "weather" in col:
        cols_to_remove.append(col)

In [ ]:
for col in df_clean.columns:
    if "deg" in col:
        cols_to_remove.append(col)

In [ ]:
for col in df_clean.columns:
    if "humidity" in col:
        cols_to_remove.append(col)

In [ ]:
keep_cols = []
for col in df_clean.columns:
    if col not in cols_to_remove:
        keep_cols.append(col)

keep_cols

In [ ]:
df_clean = df_clean[keep_cols]
df_clean.shape

In [ ]:
df_clean.head()

##### In order to prepare the data for our model, converting `seville_pressure` to a numeric feature first needs to take place. Then the data can either be normalized or standardized. This will depend on the presence of outliers. 

In [ ]:
df_clean.info()

##### Next, the `time` column is converted into datetime format for more accurate time series modelling. 

In [ ]:
df_clean['time'] = pd.to_datetime(df_clean['time'])
df_clean['time'].dtype

In [ ]:
df_clean['time_delta_hours'] = (df_clean['time'] - df_clean['time'].min()).dt.components['hours']

In [ ]:
df_clean = df_clean.drop("time", axis=1)

In [ ]:
df_clean = df_clean[['time_delta_hours'] + [col for col in df_clean.columns if col != 'time_delta_hours']]
df_clean.columns

##### Next, the features are scaled to prepare them for model training.

In [ ]:
#separating the target varable from the feature variables 
y = df_clean[target_variable]
X = df_clean.drop(target_variable, axis=1)
from sklearn.preprocessing import MinMaxScaler
# Create a MinMaxScaler object
scaler = MinMaxScaler()
# Fit the scaler to the data
scaler.fit(X)
# Transform the data
scaled_data = scaler.transform(X)
# Update the dataframe with the scaled data
df_clean[X.columns] = scaled_data
# Display the scaled data
df_clean.head()

In [ ]:
# engineer existing features for test dataset
df_clean_test = df_test
# 1. Order the columns in alphabetical order
df_clean_test = df_clean_test.reindex(sorted(df_clean_test.columns), axis=1)
# 2. Keep the "time" column in the first index position
df_clean_test = df_clean_test[['time'] + [col for col in df_clean_test.columns if col != 'time']]
# 3. Convert all column titles to lowercase
df_clean_test.columns = df_clean_test.columns.str.lower()

##### Lastly, the test dataset is put through the same preprocessing as the train dataset has been.

In [ ]:
df_clean_test['valencia_pressure'] = df_clean_test['valencia_pressure'].fillna(valencia_pressure_median)

In [ ]:
for col in df_clean_test.columns:
    if df_clean_test[col].dtype == object and col != "time":
        df_clean_test[col]= df_clean_test[col].str.extract(r'([0-9]+)')
        df_clean_test[col] = pd.to_numeric(df_clean_test[col])

In [ ]:
keep_cols_test = [col for col in keep_cols if col != target_variable]

In [ ]:
df_clean_test = df_clean_test[keep_cols_test]

In [ ]:
df_clean_test['time'] = pd.to_datetime(df_clean_test['time'])
df_clean_test['time'].dtype

##### The code snippet `df['time_delta_hours'] = (df['time'] - df['time'].min()).dt.components['hours']` calculates the time difference in hours between each timestamp in the 'TIMESTAMP' column of the DataFrame df and the minimum timestamp value in that column.

In [ ]:
df_clean_test['time_delta_hours'] = (df_clean_test['time'] - df_clean_test['time'].min()).dt.components['hours']
df_clean_test = df_clean_test.drop(['time'], axis=1)

In [ ]:
df_clean_test = df_clean_test[['time_delta_hours'] + [col for col in df_clean_test.columns if col != 'time_delta_hours']] 
df_clean_test.columns

In [ ]:
df_clean_test = df_clean_test[['time_delta_hours'] + [col for col in df_clean_test if col != 'time_delta_hours']]
#df_clean_test.columns

In [ ]:

# Create a MinMaxScaler object
scaler = MinMaxScaler()
# Fit the scaler to the data
scaler.fit(df_clean_test)
# Transform the data
scaled_data = scaler.transform(df_clean_test)
# Update the dataframe with the scaled data
df_clean_test[df_clean_test.columns] = scaled_data
# Display the scaled data
df_clean_test.head()


<a id="six"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
validations = { }

In [ ]:
dt = DecisionTreeRegressor(max_depth=5, min_samples_split=2)
dt.fit(x_train, y_train)
y_pred_val_dt = dt.predict(x_val)

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=500, random_state=42)
rf.fit(x_train, y_train.values.ravel())
y_pred_val_rf = rf.predict(x_val)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred_val_lr = lr.predict(x_val)

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge = Ridge(alpha=1.0)
ridge.fit(x_train, y_train)
y_pred_val_rd = ridge.predict(x_val)

<a id="seven"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
# Compare model performance
def rmse(y_val, y_predict):
    return np.sqrt(mean_squared_error(y_val, y_predict))

In [ ]:
# Calculate the RMSE on the validation set
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val_lr))
print("Validation Set RMSE Linear regression(LR):", rmse)

# Calculate the RMSE on the validation set
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val_rd))
print("Validation Set RMSE Linear regression (RD):", rmse)

# Calculate the RMSE on the validation set
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val_rf))
print("Validation Set RMSE Linear regression(RF):", rmse)

# Calculate the RMSE on the validation set
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val_dt))
print("Validation Set RMSE Linear regression(DT):", rmse)

In [ ]:
# Choose best model and motivate why it is the best choice

<a id="seven"></a>
## 7. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

In [ ]:
# discuss chosen methods logic